# 追加
Region false count: 983
Name false count: 1073
Name match count: 9880
Region false delim count: 2242
Name false delim count: 2377
Name match delim count: 25126

region falseを評価から除外

In [1]:
import json
import sys
sys.path.append("/home/omote/cluster_project/iam2/eval")  # Adjust the path to include the parent directory
#print(__file__.rsplit('/', 2)[0])
from eval_utils.custom_oc_cost import get_cmap,get_ot_cost,DetectedInstance
import argparse
import os
import datetime
from tqdm import tqdm
import regex as re
from torchvision.ops import box_iou
import torch
from transformers import AutoProcessor
import imgviz
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import math
from pycocotools.coco import COCO
from copy import deepcopy
from sentence_transformers import SentenceTransformer,util

/home/omote/cluster_project/iam2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def save_json(file_path, data):
    """
    Save data to a JSON file.

    Args:
        file_path (str): Path to the JSON file.
        data (dict): Data to save.
    """
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

def load_json(file_path):
    """
    Load data from a JSON file.

    Args:
        file_path (str): Path to the JSON file.

    Returns:
        dict: Data loaded from the file.
    """
    with open(file_path, 'r') as f:
        return json.load(f)

def bbox_absolute_to_relative(absolute_bbox, image_width_height):
    width, height = image_width_height
    x1 = absolute_bbox[0] / width
    y1 = absolute_bbox[1] / height
    x2 = absolute_bbox[2] / width
    y2 = absolute_bbox[3] / height
    relative_bbox = [x1, y1, x2, y2]
    return relative_bbox

def bbox_relative_to_absolute(relative_bbox, image_width_height):
    width, height = image_width_height
    x1 = relative_bbox[0] * width
    y1 = relative_bbox[1] * height
    x2 = relative_bbox[2] * width
    y2 = relative_bbox[3] * height
    absolute_bbox = [x1, y1, x2, y2]
    return absolute_bbox
    
def visualize_bbox(image, bbox_list, bbox_name_list,bbox_is_relative=True,with_id=False):
    assert len(bbox_list) == len(bbox_name_list), "bbox_list and bbox_name_list must have the same length"
    if isinstance(image, str):
        image = Image.open(image).convert("RGB")

    if bbox_is_relative:
        # 画像のサイズを取得
        image_width_height = (image.width, image.height)
        # 相対座標を絶対座標に変換
        bbox_list = [bbox_relative_to_absolute(bbox, image_width_height) for bbox in bbox_list]
        
    #bbox_name_listをソート、bbox_listも同じ順番にソート
    # bbox_name_list, bbox_list = zip(*sorted(zip(bbox_name_list, bbox_list), key=lambda x: x[0]))
    # bbox_name_list = list(bbox_name_list)
    # bbox_list = list(bbox_list)
    name_to_label_id_dict = {}
    label_id = 0
    for bbox_name in bbox_name_list:
        if bbox_name not in name_to_label_id_dict:
            name_to_label_id_dict[bbox_name] = label_id
            label_id += 1    
    
    # bbox_listの座標をy1, x1, y2, x2の形式に変換
    bboxes = []
    labels = []
    # label_id = -1
    # old_label = None
    count_object_dict = {}
    id_bbox_name_list = []
    for bbox ,bbox_name in zip(bbox_list, bbox_name_list):
        x1, y1, x2, y2 = bbox
        bboxes.append([y1, x1, y2, x2])
        # if old_label != bbox_name:
        #     label_id += 1
        #     old_label = bbox_name
        label_id = name_to_label_id_dict[bbox_name]
        if bbox_name not in count_object_dict:
            count_object_dict[bbox_name] = 0
        else:
            count_object_dict[bbox_name] += 1
        if with_id:
            bbox_name = f"{bbox_name}_{count_object_dict[bbox_name]}"
            id_bbox_name_list.append(bbox_name)
        labels.append(label_id)
    # bboxes = np.array([bbox[1],bbox[0],bbox[3],bbox[2]]).astype(np.int32).reshape(-1, 4)
    
    base_resolution = 100 * 100
    base_font_size = 3
    image_resolution = image.width * image.height
    font_size = int( base_font_size * (image_resolution / base_resolution) ** 0.5)
    
    if with_id:
        bbox_name_list = id_bbox_name_list
    image = imgviz.instances2rgb(np.array(image), bboxes=bboxes, labels=labels,font_size=font_size,captions=bbox_name_list)

    plt.imshow(image)
    plt.show()

def sort_list_of_dicts(data, key, reverse=False):
    """
    Sort a list of dictionaries by the specified key.

    Args:
        data (list): List of dictionaries to sort.
        key (str): Key to sort by.
        reverse (bool): Sort in descending order if True, ascending if False.

    Returns:
        list: Sorted list of dictionaries.
    """
    return sorted(data, key=lambda x: x[key], reverse=reverse)



In [3]:
def calculate_iou(gt_bbox_list, pred_bbox_list):
    iou_matrix = box_iou(torch.tensor(gt_bbox_list).float(), torch.tensor(pred_bbox_list).float())
    iou_matrix = torch.nan_to_num(iou_matrix, nan=0.0)  # NaNを0に置き換える
    iou_argsort_matrix = torch.argsort(iou_matrix.flatten(),descending=True).argsort().reshape(iou_matrix.shape)#iouが大きい順にソートしたインデックスを取得
    # print(iou_argsort_matrix)
    # print("-" * 50)
    # print(iou_matrix)
    pred_index_list =  torch.full((len(pred_bbox_list),), False, dtype=torch.bool)
    gt_index_list = torch.full((len(gt_bbox_list),), False, dtype=torch.bool)

    short_index_list = pred_index_list if len(pred_bbox_list) < len(gt_bbox_list) else gt_index_list
    iou_info_list = []

    # print(iou_matrix.numel())
    for i in range(iou_matrix.numel()):
        max_iou_index = torch.where(iou_argsort_matrix == i)
        if not gt_index_list[max_iou_index[0]] and not pred_index_list[max_iou_index[1]]:
            iou_info_list.append( {
                "gt_index": max_iou_index[0].item(),
                "pred_index": max_iou_index[1].item(),
                "iou_value": iou_matrix[max_iou_index].item()
            })
            gt_index_list[max_iou_index[0]] = True
            pred_index_list[max_iou_index[1]] = True
            # print(f"index {i} - gt_index: {max_iou_index[0].item()}, pred_index: {max_iou_index[1].item()}, iou_value: {iou_matrix[max_iou_index].item()}")
        
        if torch.all(short_index_list):
            break
        
    assert len(iou_info_list) == min(len(gt_bbox_list), len(pred_bbox_list)), f"Length mismatch: {len(iou_info_list)} != {min(len(gt_bbox_list), len(pred_bbox_list))}"
    # print(iou_info_list)
    # for iou_info in iou_info_list:
    #     if math.isnan(iou_info["iou_value"]):
    #         print(f"IOU value is NaN for gt index {iou_info['gt_index']} and pred index {iou_info['pred_index']}")
    #         print(iou_matrix[iou_info['gt_index'], iou_info['pred_index']])
    #         print(iou_matrix[iou_info['gt_index'], iou_info['pred_index']].item())
    #         print(iou_info["iou_value"])
    #         print(iou_matrix)
    
    return iou_info_list,iou_matrix,iou_argsort_matrix,pred_index_list, gt_index_list

def oc_cost(pred_instance_list,tgt_instance_list, alpha=0.5,beta=0.6):
    cmap_func = lambda x, y: get_cmap(x, y, alpha=alpha, beta=beta,label_or_sim="label")
    otc = get_ot_cost(pred_instance_list, tgt_instance_list, cmap_func)
    return otc

def similariry_score(str1, str2, model: SentenceTransformer):
    # compute embedding for both lists
    embedding_1 = model.encode(str1, show_progress_bar=False)
    embedding_2 = model.encode(str2, show_progress_bar=False)
    score = util.pytorch_cos_sim(embedding_1, embedding_2).item()
    
    #スコア丸め込み
    # score = min(score, 1.0)
    # score = max(score, 0.0)
    
    return score

def create_get_most_similar_category_func(category_list, sentence_transformer_model_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    similarity_model = SentenceTransformer(sentence_transformer_model_path).to(device)
    category_embeddings = similarity_model.encode(category_list, show_progress_bar=False, convert_to_tensor=True)
    def get_most_similar_category(category_name):
        category_embedding = similarity_model.encode(category_name, show_progress_bar=False, convert_to_tensor=True)
        scores = util.pytorch_cos_sim(category_embedding, category_embeddings).squeeze(0)
        most_similar_index = torch.argmax(scores).item()
        return category_list[most_similar_index], scores[most_similar_index].item(), scores
    return get_most_similar_category

In [102]:
get_most_similar_category = create_get_most_similar_category_func(
    ["airplane", "bicycle", "bird", "boat"],
    "/data_ssd/huggingface_model_weights/sentence-transformers/all-MiniLM-L6-v2"
)

In [103]:
category, score, scores = get_most_similar_category("plane")
print(f"Best match category: {category}, score: {score}")

Best match category: airplane, score: 0.8521708250045776


In [ ]:
def get_per_image_class_result_and_oc_cost(all_gt_annotations, all_pred_annotations, cat_name2id, iou_threshold=0.5):
    """
    Evaluate per-image and per-category results and calculate occlusion cost.

    Args:
        all_gt_annotations (dict): Ground truth annotations.
        all_pred_annotations (dict): Predicted annotations.
        cat_name2id (dict): Category name to ID mapping.
        iou_threshold (float): IOU threshold for true positives.

    Returns:
        dict: Per-image results and occlusion costs.
    """
    per_image_result_dict = {}
    break_index = 10
    oc_cost_list = []
    for index, gt_per_image_annotation_list in tqdm(all_gt_annotations.items()):
        pred_per_image_annotation_list = all_pred_annotations.get(index, [])
        
        # 画像ごとの評価
        pred_instance_list = [DetectedInstance(
            label=ann["category_id"],
            x1=ann["bbox_xyxy"][0],
            y1=ann["bbox_xyxy"][1],
            x2=ann["bbox_xyxy"][2],
            y2=ann["bbox_xyxy"][3]) for ann in pred_per_image_annotation_list]
        tgt_instance_list = [DetectedInstance(
            label=ann["category_id"],
            x1=ann["bbox_xyxy"][0],
            y1=ann["bbox_xyxy"][1],
            x2=ann["bbox_xyxy"][2],
            y2=ann["bbox_xyxy"][3]) for ann in gt_per_image_annotation_list]
        
        oc_cost_value = oc_cost(pred_instance_list, tgt_instance_list, alpha=0.5, beta=0.6)
        oc_cost_list.append(oc_cost_value)
        
        #画像ごと・カテゴリごとの評価準備
        gt_per_category_dict = {}
        pred_per_category_dict = {}
        per_category_result_dict = {}
        
        for category_id in cat_name2id.values():
            gt_per_category_dict[category_id] = None
            pred_per_category_dict[category_id] = None
            per_category_result_dict[category_id] = None
            
        for annotation in gt_per_image_annotation_list:
            if gt_per_category_dict[annotation["category_id"]] is None:
                gt_per_category_dict[annotation["category_id"]] = []
            gt_per_category_dict[annotation["category_id"]].append(annotation)
        
        for annotation in pred_per_image_annotation_list:
            if pred_per_category_dict[annotation["category_id"]] is None:
                pred_per_category_dict[annotation["category_id"]] = []
            pred_per_category_dict[annotation["category_id"]].append(annotation)
        
        
        for category_id, gt_annotations in gt_per_category_dict.items():
            pred_annotations = pred_per_category_dict[category_id]
            if gt_annotations is None and  pred_annotations is None:
                continue
            
            per_category_result = {
                "iou_list": [],
                "pred_iou_list": [],
                "tp_num": 0,
                "fp_num": 0,
                "fn_num": 0,
            }
            if gt_annotations is None and pred_per_category_dict[category_id] is not None:
                per_category_result["fp_num"] = len(pred_per_category_dict[category_id])
            elif gt_annotations is not None:
                if pred_per_category_dict[category_id] is None:
                    per_category_result["fn_num"] = len(gt_annotations)
                    per_category_result["iou_list"] = [0.0] * len(gt_annotations)
                else: 
                    gt_bbox_list = [ann["bbox_xyxy"] for ann in gt_annotations]
                    pred_bbox_list = [ann["bbox_xyxy"] for ann in pred_annotations]
                    iou_info_list,iou_matrix,iou_argsort_matrix,pred_index_list, gt_index_listt = calculate_iou(gt_bbox_list, pred_bbox_list)
                    assert ((len(gt_bbox_list) < len(pred_bbox_list) and len(iou_info_list) == len(gt_bbox_list)) or (len(gt_bbox_list) >= len(pred_bbox_list) and len(iou_info_list) == len(pred_bbox_list))), f"Length mismatch in category {category_id}, index {index}: len(iou_info_list)={len(iou_info_list)}, len(gt_bbox_list)={len(gt_bbox_list)}, len(pred_bbox_list)={len(pred_bbox_list)}"
                    # if not((len(gt_bbox_list) < len(pred_bbox_list) and len(iou_info_list) == len(gt_bbox_list)) or \
                    #     (len(gt_bbox_list) >= len(pred_bbox_list) and len(iou_info_list) == len(pred_bbox_list))):
                        # print(f"index: {index}, category_id: {category_id}, len(iou_info_list): {len(iou_info_list)}, len(gt_bbox_list): {len(gt_bbox_list)}, len(pred_bbox_list): {len(pred_bbox_list)}")
                        # print(f"pred_bbox_list: {pred_bbox_list}")
                        # print(f"gt_bbox_list: {gt_bbox_list}")
                        # print(f"iou_info_list: {iou_info_list}")
                        # print(f"iou_matrix: {iou_matrix}")
                        # print(f"iou_argsort_matrix: {iou_argsort_matrix}")
                        # print(f"pred_index_list: {pred_index_list}")
                        # print(f"gt_index_list: {gt_index_listt}")
                        # raise ValueError("IOU information length mismatch")
                    iou_list = [info["iou_value"] for info in iou_info_list]
                    per_category_result["pred_iou_list"] = deepcopy(iou_list)
                    for iou in iou_list:
                        assert not math.isnan(iou), f"IOU value is NaN in category {category_id}, index {index}"
                    if len(iou_list) < len(gt_bbox_list):
                        iou_list += [0.0] * (len(gt_bbox_list) - len(iou_list))
                    
                    
                    # for iou in iou_list:
                    #     assert not math.isnan(iou), f"IOU value is NaN in category {category_id}, index {index}"
                    per_category_result["iou_list"] = iou_list
                    tp_num = sum(1 for iou in iou_list if iou >= iou_threshold)
                    per_category_result["tp_num"] = tp_num
                    per_category_result["fp_num"] = len(pred_bbox_list) - tp_num
                    per_category_result["fn_num"] = len(gt_bbox_list) - tp_num
                    # if index == 9 and category_id == 84:
                    #     visualize_bbox(
                    #         os.path.join(image_folder_root, images[index]["file_name"]),
                    #         pred_bbox_list,
                    #         [ann["category_name"] for ann in pred_annotations],
                    #         bbox_is_relative=True,
                    #         with_id=True
                    #     )
                    #     print(per_category_result)
                    #     print(pred_bbox_list == gt_bbox_list)
                    #     print(len(pred_bbox_list), len(gt_bbox_list))
                    #     print(iou_info_list)
            
            per_category_result_dict[category_id] = per_category_result
        
        per_image_result_dict[index] = per_category_result_dict
        # if index >= break_index:
        #     break
    return per_image_result_dict, oc_cost_list 

def convert_per_class_result_dict(per_image_result_dict):
    per_category_result_dict = {}
    for index, per_image_result in per_image_result_dict.items():
        for category_id, result in per_image_result.items():
            if category_id not in per_category_result_dict:
                per_category_result_dict[category_id] = {
                    "iou_list": [],
                    "pred_iou_list": [],
                    "tp_num": 0,
                    "fp_num": 0,
                    "fn_num": 0,
                }
            
            if result is None:
                continue
            # if result["fp_num"] > 0 or result["fn_num"]:
            #     print(index, category_id, result)
            per_category_result_dict[category_id]["tp_num"] += result["tp_num"]
            per_category_result_dict[category_id]["fp_num"] += result["fp_num"]
            per_category_result_dict[category_id]["fn_num"] += result["fn_num"]
            per_category_result_dict[category_id]["iou_list"].extend(result["iou_list"])
            per_category_result_dict[category_id]["pred_iou_list"].extend(result["pred_iou_list"])
    return per_category_result_dict

def calculate_score(per_category_result_dict,oc_cost_list,category_id2name):
    """
    Calculate precision, recall, F1 score, and mean IOU for each category and overall dataset.

    Args:
        per_category_result_dict (dict): Per-category results.
        oc_cost_list (list): List of occlusion costs.

    Returns:
        dict: Summary scores and data numbers for the dataset.
    """
    per_category_score_dict = {}

    dataset_score = {
        "summary_scores":{
            "micro_precision": 0.0,
            "micro_recall": 0.0,
            "micro_f1": 0.0,
            "m_iou": [],
            "m_pred_iou": [],
            "oc_cost": np.mean(oc_cost_list) if len(oc_cost_list) > 0 else 0.0,
            "macro_precision": [],
            "macro_recall": [],
            "macro_f1": [],
            "cm_iou": [],
            "cm_pred_iou": [],
        },
        "summary_data_num":{
            "tp_num": 0,
            "fp_num": 0,
            "fn_num": 0,
            "unkonown_fp_num": per_category_result_dict[-1]["fp_num"] if -1 in per_category_result_dict else 0,
            "iou_num": 0,
            "pred_iou_num": 0,
        },
    }

    for category_id, result in per_category_result_dict.items():
        #クラスごと
        tp_num = result["tp_num"]
        fp_num = result["fp_num"]
        fn_num = result["fn_num"]
        iou_list = result["iou_list"]
        pred_iou_list = result["pred_iou_list"]
    
        cm_iou = np.mean(iou_list) if len(iou_list) > 0 else 0.0
        cm_pred_iou = np.mean(pred_iou_list) if len(pred_iou_list) > 0 else 0.0
        # if math.isnan(m_iou):
        #     print(f"Category ID: {category_id} has NaN mIoU. Check the IOU list: {iou_list}")
        #     for iou in iou_list:
        #         assert not math.isnan(iou), f"IOU value is NaN in category {category_id}"
        precision = tp_num / (tp_num + fp_num) if (tp_num + fp_num) > 0 else 0.0
        recall = tp_num / (tp_num + fn_num) if (tp_num + fn_num) > 0 else 0.0
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
        per_category_score = {
            "category_name": category_id2name[category_id],
            "precision": precision,
            "recall": recall,
            "f1_score": f1_score,
            "cm_iou": cm_iou,
            "cm_pred_iou": cm_pred_iou,
            "tp_num": tp_num,
            "fp_num": fp_num,
            "fn_num": fn_num,
        }
        per_category_score_dict[category_id] = per_category_score
        
        #データセット全体
        dataset_score["summary_data_num"]["tp_num"] += tp_num
        dataset_score["summary_data_num"]["fp_num"] += fp_num
        dataset_score["summary_data_num"]["fn_num"] += fn_num
        dataset_score["summary_data_num"]["iou_num"] += len(iou_list)
        dataset_score["summary_data_num"]["pred_iou_num"] += len(pred_iou_list)
        dataset_score["summary_scores"]["m_iou"].extend(iou_list)
        dataset_score["summary_scores"]["m_pred_iou"].extend(pred_iou_list)
        
        #カテゴリごと
        if category_id != -1:
            dataset_score["summary_scores"]["macro_precision"].append(precision)
            dataset_score["summary_scores"]["macro_recall"].append(recall)
            dataset_score["summary_scores"]["macro_f1"].append(f1_score)
            dataset_score["summary_scores"]["cm_iou"].append(cm_iou)
            dataset_score["summary_scores"]["cm_pred_iou"].append(cm_pred_iou)

    #データセット全体
    assert dataset_score["summary_data_num"]["tp_num"] + dataset_score["summary_data_num"]["fn_num"] == len(dataset_score["summary_scores"]["m_iou"]) ,\
        f"TP + FN mismatch: {dataset_score['summary_data_num']['tp_num']} + {dataset_score['summary_data_num']['fn_num']} != {len(dataset_score['summary_scores']['m_iou'])}"
    dataset_score["summary_scores"]["micro_precision"] = dataset_score["summary_data_num"]["tp_num"] / (dataset_score["summary_data_num"]["tp_num"] + dataset_score["summary_data_num"]["fp_num"]) if (dataset_score["summary_data_num"]["tp_num"] + dataset_score["summary_data_num"]["fp_num"]) > 0 else 0.0
    dataset_score["summary_scores"]["micro_recall"] = dataset_score["summary_data_num"]["tp_num"] / (dataset_score["summary_data_num"]["tp_num"] + dataset_score["summary_data_num"]["fn_num"]) if (dataset_score["summary_data_num"]["tp_num"] + dataset_score["summary_data_num"]["fn_num"]) > 0 else 0.0
    dataset_score["summary_scores"]["micro_f1"] = (2 * dataset_score["summary_scores"]["micro_precision"] * dataset_score["summary_scores"]["micro_recall"]) / (dataset_score["summary_scores"]["micro_precision"] + dataset_score["summary_scores"]["micro_recall"]) if (dataset_score["summary_scores"]["micro_precision"] + dataset_score["summary_scores"]["micro_recall"]) > 0 else 0.0
    dataset_score["summary_scores"]["m_iou"] = np.mean(dataset_score["summary_scores"]["m_iou"]) if len(dataset_score["summary_scores"]["m_iou"]) > 0 else 0.0
    dataset_score["summary_scores"]["m_pred_iou"] = np.mean(dataset_score["summary_scores"]["m_pred_iou"]) if len(dataset_score["summary_scores"]["m_pred_iou"]) > 0 else 0.0
    dataset_score["summary_scores"]["macro_precision"] = np.mean(dataset_score["summary_scores"]["macro_precision"]) if len(dataset_score["summary_scores"]["macro_precision"]) > 0 else 0.0
    dataset_score["summary_scores"]["macro_recall"] = np.mean(dataset_score["summary_scores"]["macro_recall"]) if len(dataset_score["summary_scores"]["macro_recall"]) > 0 else 0.0
    dataset_score["summary_scores"]["macro_f1"] = np.mean(dataset_score["summary_scores"]["macro_f1"]) if len(dataset_score["summary_scores"]["macro_f1"]) > 0 else 0.0
    dataset_score["summary_scores"]["cm_iou"] = np.mean(dataset_score["summary_scores"]["cm_iou"]) if len(dataset_score["summary_scores"]["cm_iou"]) > 0 else 0.0 
    dataset_score["summary_scores"]["cm_pred_iou"] = np.mean(dataset_score["summary_scores"]["cm_pred_iou"]) if len(dataset_score["summary_scores"]["cm_pred_iou"]) > 0 else 0.0
    
    for category_id, score in per_category_score_dict.items():
        print(f"category_name: {score['category_name']}")
        print(f"Category ID: {category_id}, Precision: {score['precision']:.4f}, Recall: {score['recall']:.4f}, F1 Score: {score['f1_score']:.4f}, cmIoU: {score['cm_iou']:.4f}, cmPredIoU: {score['cm_pred_iou']:.4f}, TP: {score['tp_num']}, FP: {score['fp_num']}, FN: {score['fn_num']}")

    for key,score in dataset_score["summary_scores"].items():
        print(f"{key}: {score:.4f}")
        
    for key,num in dataset_score["summary_data_num"].items():
        print(f"{key}: {num}")
    return per_category_score_dict, dataset_score

In [ ]:
correct_json_path = "/data_ssd/PASCAL-VOC/val_pascal-voc-one-class_for-kosmos2_one_class_with_delim.json"
correct_data = load_json(correct_json_path)

generated_json_path = "/data_ssd/PASCAL-VOC/val_pascal-voc-one-class_for-kosmos2_one_class_with_delim.json"
#generated_json_path = "/home/omote/omote-data-ssd/iam-llms-finetune/experiment_output/kosmos-2_pascalvoc-one-class/with-delim_train-vision-proj-llm_cross-entropy_2025-07-14T16_51_39/checkpoint-6144/eval_output/val_pascal-voc-one-class_for-kosmos2_one_class_with_delim/max_new_tokens=512-temperature=1.0-top_p=1.0-top_k=50-num_beams=1-do_sample=False-/2025-07-14T19_28_38/eval_output.json"
#generated_json_path = "/home/omote/omote-data-ssd/iam-llms-finetune/experiment_output/kosmos-2_pascalvoc-one-class/without-delim_train-vision-proj-llm_cross-entropy_2025-07-15T08_16_57/checkpoint-6144/eval_output/val_pascal-voc-one-class_for-kosmos2_one_class_without_delim/max_new_tokens=512-temperature=1.0-top_p=1.0-top_k=50-num_beams=1-do_sample=False-/2025-07-15T10_14_11/eval_output.json"
generated_json_path = "/home/omote/omote-data-ssd/iam-llms-finetune/experiment_output/kosmos-2_pascalvoc-one-class/without-delim-noline_include-input-tokens_train-vision-proj-llm_cross-entropy_2025-07-15T16_35_34/checkpoint-6144/eval_output/val_pascal-voc-one-class_for-kosmos2_one_class_without_delim_noline/max_new_tokens=512-temperature=1.0-top_p=1.0-top_k=50-num_beams=1-do_sample=False-/2025-07-15T18_32_42/eval_output.json"
generated_data = load_json(generated_json_path)

assert len(correct_data) == len(generated_data), "Length of correct and generated data does not match."

correct_data = sort_list_of_dicts(correct_data, "id")
generated_data = sort_list_of_dicts(generated_data, "id")

for correct, generated in zip(correct_data, generated_data):
    assert correct["id"] == generated["id"], f"ID mismatch: {correct['id']} != {generated['id']}"

In [95]:
ann_id_list = []
for correct in correct_data:
    ann_id_list.append(correct["ann_id"])

print(len(ann_id_list))
print(len(set(ann_id_list)))

3618
2510


In [96]:
def get_pascal_voc_category():
    cat_id2name = {
    0: "aeroplane",
    1: "bicycle",
    2: "bird",
    3: "boat",
    4: "bottle",
    5: "bus",
    6: "car",
    7: "cat",
    8: "chair",
    9: "cow",
    10: "diningtable",
    11: "dog",
    12: "horse",
    13: "motorbike",
    14: "person",
    15: "pottedplant",
    16: "sheep",
    17: "sofa",
    18: "train",
    19: "tvmonitor",
    }
    
    cat_name2id = {v:k for k, v in cat_id2name.items()}
    return cat_name2id, cat_id2name

In [97]:
category_name2id,category_id2name = get_pascal_voc_category()
# category_name2id.update({"unknown": -1})
# category_id2name.update({-1: "unknown"})
processor = AutoProcessor.from_pretrained("/data_ssd/huggingface_model_weights/microsoft/kosmos-2-patch14-224")

In [98]:
def create_annotations_for_coco(conversation_dataset,categories,processor,delete_region_failure=False,unknown_to_similar=False,sentence_transformer_model_path=None):
    if unknown_to_similar and not delete_region_failure:
        raise ValueError("unknown_to_similar is True but delete_region_failure is False. This combination is not supported.")
    elif unknown_to_similar and delete_region_failure:
        get_most_similar_category = create_get_most_similar_category_func(
            list(categories.keys()),
            sentence_transformer_model_path
        )
    else:
        get_most_similar_category = None

    ann_id_converastaion_dict = {}
    for i, conversation in enumerate(conversation_dataset):
        if conversation["ann_id"] not in ann_id_converastaion_dict:
            ann_id_converastaion_dict[conversation["ann_id"]] = []
        ann_id_converastaion_dict[conversation["ann_id"]].append(i)
        
    return_annotations = {}
    
    ann_keys_list = ann_id_converastaion_dict.keys()

    region_failure_count = 0
    region_failure_delim_count = 0
    name_failure_count = 0
    name_failure_delim_count = 0
    name_match_count = 0
    name_match_delim_count = 0

    id_index = 0
    for i, ann_key in enumerate(tqdm(ann_keys_list)):
        for conversation in ann_id_converastaion_dict[ann_key]:
            text = ""
            for conv in conversation_dataset[conversation]["conversations"]:
                text += conv["value"]

            caption, entities = processor.post_process_generation(text)
            for name,_,bbox_list in entities:
                if "<patch_index" in name and delete_region_failure:
                    #raise ValueError(f"Unexpected patch index in name: {name}")
                    region_failure_count += 1
                    region_failure_delim_count += len(bbox_list)
                    continue
                elif name not in categories.keys():    
                    name_failure_count += 1
                    name_failure_delim_count += len(bbox_list)
                    if get_most_similar_category is not None:
                        name, score, _ = get_most_similar_category(name)
                else:
                    name_match_count += 1
                    name_match_delim_count += len(bbox_list)
                    
                for bbox in bbox_list:
                    annotation = {
                        "id": id_index,
                        "image_id": i ,
                        "category_id": categories[name] if name in categories else categories["unknown"],
                        "bbox": [bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]],  # [x, y, width, height]
                        "area": (bbox[2] - bbox[0]) * (bbox[3] - bbox[1]),
                        "iscrowd": 0,
                        "score": 1.0,  # Assuming all annotations are perfect for dummy data
                        "category_name": name,
                        "bbox_xyxy": bbox,  # [x1, y1, x2, y2]
                        "is_unknown": 1 if name not in categories else 0
                    }
                    
                    if i not in return_annotations:
                        return_annotations[i] = []
                    return_annotations[i].append(annotation)
                    id_index += 1

    return_num_dict = {
        "region_failure_count": region_failure_count,
        "region_failure_delim_count": region_failure_delim_count,
        "name_failure_count": name_failure_count,
        "name_failure_delim_count": name_failure_delim_count,
        "name_match_count": name_match_count,
        "name_match_delim_count": name_match_delim_count
    }
    return return_annotations, return_num_dict

In [99]:
all_gt_annotations, all_gt_num_dict = create_annotations_for_coco(correct_data, category_name2id, processor)
all_pred_annotations, all_pred_num_dict = create_annotations_for_coco(generated_data, category_name2id, processor,
            delete_region_failure=True, unknown_to_similar=True, sentence_transformer_model_path="/data_ssd/huggingface_model_weights/sentence-transformers/all-MiniLM-L6-v2")

print(len(all_gt_annotations))
print(len(all_pred_annotations))
for key, value in all_gt_num_dict.items():
    print(f"GT {key}: {value}")
for key, value in all_pred_num_dict.items():
    print(f"Generated {key}: {value}")

100%|██████████| 2510/2510 [00:07<00:00, 330.76it/s]

2510
2510
GT region_failure_count: 0
GT region_failure_delim_count: 0
GT name_failure_count: 0
GT name_failure_delim_count: 0
GT name_match_count: 6307
GT name_match_delim_count: 6307
Generated region_failure_count: 248
Generated region_failure_delim_count: 248
Generated name_failure_count: 1243
Generated name_failure_delim_count: 1243
Generated name_match_count: 3625
Generated name_match_delim_count: 3625


In [100]:
# for anno in all_generated_annotations.values():
#     for a in anno:
#         if a["is_unknown"] == 1:
#             print(a["category_name"])
#             print(a["bbox_xyxy"])
#             print(a["bbox"])
#             print(a["bbox_xyxy"][2] - a["bbox_xyxy"][0])
#             print(a["bbox_xyxy"][3] - a["bbox_xyxy"][1])
#             print("-" * 50)
#             break

In [101]:
per_image_result_dict, oc_cost_list = get_per_image_class_result_and_oc_cost(all_gt_annotations, all_pred_annotations, category_name2id, iou_threshold=0.5)
per_category_result_dict = convert_per_class_result_dict(per_image_result_dict)

_,output_data = calculate_score(per_category_result_dict, oc_cost_list,category_id2name)

100%|██████████| 2510/2510 [00:01<00:00, 1278.95it/s]

category_name: aeroplane
Category ID: 0, Precision: 0.9007, Recall: 0.8194, F1 Score: 0.8581, cmIoU: 0.7671, cmPredIoU: 0.8554, TP: 127, FP: 14, FN: 28
category_name: bicycle
Category ID: 1, Precision: 0.8294, Recall: 0.7966, F1 Score: 0.8127, cmIoU: 0.7097, cmPredIoU: 0.8052, TP: 141, FP: 29, FN: 36
category_name: bird
Category ID: 2, Precision: 0.8325, Recall: 0.6749, F1 Score: 0.7455, cmIoU: 0.6082, cmPredIoU: 0.7946, TP: 164, FP: 33, FN: 79
category_name: boat
Category ID: 3, Precision: 0.7955, Recall: 0.7000, F1 Score: 0.7447, cmIoU: 0.6359, cmPredIoU: 0.7692, TP: 105, FP: 27, FN: 45
category_name: bottle
Category ID: 4, Precision: 0.5259, Recall: 0.2817, F1 Score: 0.3669, cmIoU: 0.2545, cmPredIoU: 0.5214, TP: 71, FP: 64, FN: 181
category_name: bus
Category ID: 5, Precision: 0.8922, Recall: 0.7982, F1 Score: 0.8426, cmIoU: 0.7444, cmPredIoU: 0.8748, TP: 91, FP: 11, FN: 23
category_name: car
Category ID: 6, Precision: 0.7864, Recall: 0.6128, F1 Score: 0.6888, cmIoU: 0.5574, cmPredI

In [ ]:
#

# without-delim-noline
# with unknown no sim
micro_precision: 0.6588
micro_recall: 0.5085
micro_f1: 0.5740
m_iou: 0.4632
m_pred_iou: 0.8062
oc_cost: 0.1699
macro_precision: 0.8736
macro_recall: 0.6151
macro_f1: 0.7101
cm_iou: 0.5638
cm_pred_iou: 0.8007
tp_num: 3207
fp_num: 1661
fn_num: 3100
unkonown_fp_num: 1243
iou_num: 6307
pred_iou_num: 3624

# with unknown with sim
micro_precision: 0.8209
micro_recall: 0.6336
micro_f1: 0.7152
m_iou: 0.5752
m_pred_iou: 0.7878
oc_cost: 0.1183
macro_precision: 0.8111
macro_recall: 0.7149
macro_f1: 0.7539
cm_iou: 0.6529
cm_pred_iou: 0.7872
tp_num: 3996
fp_num: 872
fn_num: 2311
unkonown_fp_num: 9
iou_num: 6307
pred_iou_num: 4605

# no unknown with sim
micro_precision: 0.8213
micro_recall: 0.6339
micro_f1: 0.7155
m_iou: 0.5755
m_pred_iou: 0.7878
oc_cost: 0.1183
macro_precision: 0.8101
macro_recall: 0.7150
macro_f1: 0.7535
cm_iou: 0.6530
cm_pred_iou: 0.7872
tp_num: 3998
fp_num: 870
fn_num: 2309
unkonown_fp_num: 0
iou_num: 6307
pred_iou_num: 4607

# with-delim
# with unkonown no sim
micro_precision: 0.7827
micro_recall: 0.7810
micro_f1: 0.7818
m_iou: 0.7047
m_pred_iou: 0.7666
oc_cost: 0.0840
macro_precision: 0.7953
macro_recall: 0.8050
macro_f1: 0.7989
cm_iou: 0.7296
cm_pred_iou: 0.7780
tp_num: 4926
fp_num: 1368
fn_num: 1381
unkonown_fp_num: 5
iou_num: 6307
pred_iou_num: 5798

# with unkonown with sim
micro_precision: 0.7827
micro_recall: 0.7810
micro_f1: 0.7818
m_iou: 0.7047
m_pred_iou: 0.7666
oc_cost: 0.0840
macro_precision: 0.7953
macro_recall: 0.8050
macro_f1: 0.7989
cm_iou: 0.7296
cm_pred_iou: 0.7780
tp_num: 4926
fp_num: 1368
fn_num: 1381
unkonown_fp_num: 5
iou_num: 6307
pred_iou_num: 5798

# no unknown with sim
micro_precision: 0.7827
micro_recall: 0.7810
micro_f1: 0.7818
m_iou: 0.7047
m_pred_iou: 0.7666
oc_cost: 0.0840
macro_precision: 0.7947
macro_recall: 0.8050
macro_f1: 0.7986
cm_iou: 0.7296
cm_pred_iou: 0.7780
tp_num: 4926
fp_num: 1368
fn_num: 1381
unkonown_fp_num: 0
iou_num: 6307
pred_iou_num: 5798


# without-delim
# with unknown no sim
micro_precision: 0.6347
micro_recall: 0.5245
micro_f1: 0.5744
m_iou: 0.4754
m_pred_iou: 0.8018
oc_cost: 0.1633
macro_precision: 0.8586
macro_recall: 0.6327
macro_f1: 0.7169
cm_iou: 0.5760
cm_pred_iou: 0.7994
tp_num: 3308
fp_num: 1904
fn_num: 2999
unkonown_fp_num: 1383
iou_num: 6307
pred_iou_num: 3740

# with unknown with sim
micro_precision: 0.7972
micro_recall: 0.6588
micro_f1: 0.7214
m_iou: 0.5945
m_pred_iou: 0.7867
oc_cost: 0.1114
macro_precision: 0.8043
macro_recall: 0.7183
macro_f1: 0.7532
cm_iou: 0.6519
cm_pred_iou: 0.7889
tp_num: 4155
fp_num: 1057
fn_num: 2152
unkonown_fp_num: 84
iou_num: 6307
pred_iou_num: 4766


# no unknown with sim
micro_precision: 0.7972
micro_recall: 0.6588
micro_f1: 0.7214
m_iou: 0.5945
m_pred_iou: 0.7864
oc_cost: 0.1114
macro_precision: 0.7942
macro_recall: 0.7183
macro_f1: 0.7486
cm_iou: 0.6519
cm_pred_iou: 0.7889
tp_num: 4155
fp_num: 1057
fn_num: 2152
unkonown_fp_num: 0
iou_num: 6307
pred_iou_num: 4768
